# AI Trades Bot — Mobile Controller (Google Colab)
Use this on your iPhone/iPad via the Colab app or browser. It runs the bot in the Colab cloud.
**Tip:** Keep the tab open to keep it running; for long sessions, deploy the Streamlit app instead.

In [ ]:
# 1) Setup: clone/upload your bot repo (or upload the ZIP from earlier)
import os, shutil, zipfile, sys, subprocess, glob

#@title Upload your ai_trades_bot_v2.zip
from google.colab import files
print("Upload ai_trades_bot_v2.zip (from our chat downloads)")
up = files.upload()

zip_name = [k for k in up.keys() if k.endswith('.zip')][0]
workdir = '/content/ai_trades_bot'
if os.path.isdir(workdir):
    shutil.rmtree(workdir)
os.makedirs(workdir, exist_ok=True)
with zipfile.ZipFile(zip_name, 'r') as z:
    z.extractall(workdir)

print('Extracted to', workdir)
%cd /content/ai_trades_bot
!ls -la


In [ ]:
# 2) Install requirements
!pip -q install -r requirements.txt

In [ ]:
# 3) Configure run parameters
SYMBOLS = "AAPL,TSLA" #@param {type:"string"}
STRATEGY = "orb_breakout" #@param ["gap_and_go","orb_breakout","vwap_reversion"]
ASSET_CLASS = "stock" #@param ["stock","crypto"]
MODE = "paper" #@param ["paper","live"]
BROKER = "" #@param ["","alpaca","binance","bybit","kraken"]
RISK = 0.003 #@param {type:"number"}
INTERVAL = "1m" #@param ["1m","5m","15m"]
PARTIALS = "50@1R,50@2R" #@param {type:"string"}
TRAIL_PCT = 0.005 #@param {type:"number"}
NOTIFY = True #@param {type:"boolean"}

# Optional: Discord/Telegram secrets (only if NOTIFY is True)
DISCORD_WEBHOOK_URL = "" #@param {type:"string"}
TELEGRAM_BOT_TOKEN = "" #@param {type:"string"}
TELEGRAM_CHAT_ID = "" #@param {type:"string"}

import os
if DISCORD_WEBHOOK_URL: os.environ['DISCORD_WEBHOOK_URL'] = DISCORD_WEBHOOK_URL
if TELEGRAM_BOT_TOKEN: os.environ['TELEGRAM_BOT_TOKEN'] = TELEGRAM_BOT_TOKEN
if TELEGRAM_CHAT_ID: os.environ['TELEGRAM_CHAT_ID'] = TELEGRAM_CHAT_ID

cmd = [
    "python","main.py",
    "--mode", MODE,
    "--symbols", SYMBOLS,
    "--strategy", STRATEGY,
    "--asset_class", ASSET_CLASS,
    "--risk", str(RISK),
    "--interval", INTERVAL,
    "--partials", PARTIALS,
    "--trail_pct", str(TRAIL_PCT),
]
if BROKER:
    cmd += ["--broker", BROKER]
if NOTIFY:
    cmd += ["--notify"]

print(" ".join(cmd))


In [ ]:
# 4) Run once (tap Play to execute)
import subprocess, sys, os
res = subprocess.run(cmd, capture_output=False)
print("Done.")

In [ ]:
# 5) Loop-run (runs every 60s; stop the cell to end)
import time, subprocess, sys

while True:
    print("\n=== Tick ===")
    subprocess.run(cmd, check=False)
    time.sleep(60)
